# <center> **AMAZON SENTIMENT ANALYSIS**

<center> Project by Rishita Chouhan, Sakina Ariwala, Lalita Naidu, Viha Sharma

# Importing dependencies

In [ ]:
 from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import gzip
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

import nltk
from nltk.corpus import stopwords
nltk.download('all')

pd.set_option('display.max_colwidth', 200)

# Data acquisiton


In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
meta_hk = getDF('/content/drive/MyDrive/meta_Home_and_Kitchen.json.gz')
meta_hk.head()

,asin,salesRank,imUrl,categories,title,description,related,price,brand
0,0076144011,{'Books': 6285595},http://g-ecx.images-amazon.com/images/G/01/x-s...,[[Home & Kitchen]],"Ninjas, Piranhas, and Galileo",NaN,NaN,NaN,NaN
1,0130350591,{'Kitchen & Dining': 459680},http://ecx.images-amazon.com/images/I/21zcx6RC...,"[[Home & Kitchen, Kitchen & Dining, Dining & E...",Le Creuset Kiwi (Green) Butter Dish Stoneware,Each piece of Le Creuset dinnerware is crafted...,NaN,NaN,NaN
2,0307394530,"{'Arts, Crafts & Sewing': 3597}",http://ecx.images-amazon.com/images/I/51A4FWuj...,"[[Home & Kitchen, Artwork, Posters & Prints]]",Martha Stewart's Wedding Cakes,Of all the decisions that go into planning a w...,"{'also_bought': ['144630163X', '1402717733', '...",14.99,Random House
3,0439903491,{'Software': 7065},http://ecx.images-amazon.com/images/I/61tVxcko...,"[[Home & Kitchen, Artwork, Posters & Prints]]",NaN,Shiver me timbers! Solve I SPY pirate picture ...,"{'also_viewed': ['B000YFSZU8', 'B0016KWZB0', '...",29.99,NaN
4,0578060604,NaN,http://ecx.images-amazon.com/images/I/41aCELWJ...,"[[Home & Kitchen, Furniture, Kitchen & Dining ...",Build A Maloof Inspired Low Back Dining Chair ...,The Maloofinspired Low Back Dining Chair is no...,{'also_viewed': ['B004IO6RS8']},NaN,NaN


In [ ]:
meta_music = getDF('/content/drive/MyDrive/meta_Musical_Instruments.json.gz')
meta_music.head()

,asin,title,price,imUrl,salesRank,categories,related,brand,description
0,0006428320,"Six Sonatas For Two Flutes Or Violins, Volume ...",17.95,http://ecx.images-amazon.com/images/I/41EpRmh8...,{'Musical Instruments': 207315},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,NaN
1,0014072149,Double Concerto in D Minor By Johann Sebastian...,18.77,http://ecx.images-amazon.com/images/I/41m6ygCq...,{'Musical Instruments': 94593},[[Musical Instruments]],"{'also_viewed': ['B0058DK7RA'], 'buy_after_vie...",,Composer: J.S. Bach.Peters Edition.For two vio...
2,0041291905,Hal Leonard Vivaldi Four Seasons for Piano (Or...,NaN,http://ecx.images-amazon.com/images/I/41maAqSO...,{'Musical Instruments': 222972},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,Vivaldi's famous set of four violin concertos ...
3,0041913574,"Aida: Opera in Quattro Atti, Partitura -- Aida...",49.99,http://ecx.images-amazon.com/images/I/513kRMv%...,NaN,[[Musical Instruments]],NaN,NaN,444 pages. \nReprint of corrected and revised ...
4,0201891859,Nocturnes,NaN,http://ecx.images-amazon.com/images/I/41SXCAzs...,{'Musical Instruments': 171871},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,NaN


In [ ]:
review_hk = getDF('/content/drive/MyDrive/reviews_Home_and_Kitchen.json.gz')
review_hk.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A210NOCSTBT4OD,0076144011,Sheila,"[0, 0]",Have you ever thought about how you met your b...,4.0,Lovely,1349308800,"10 4, 2012"
1,A28ILV4TOG8BH2,0130350591,ccjensen,"[0, 0]","The butter dish is serving us well, and keepin...",5.0,"Nice looking, and keeps the butter fresh",1300752000,"03 22, 2011"
2,A31B4D7URW4DNZ,0307394530,3Gigi3,"[11, 16]",I anxiously waited for the book I had pre orde...,2.0,Mother of the Bride,1214784000,"06 30, 2008"
3,A2HU0RPDRZZOP1,0307394530,Alexey Leontev,"[0, 0]","Fantastic book, a lot of good, original recipe...",5.0,amazing book,1277337600,"06 24, 2010"
4,A7J0XOW7DYBBD,0307394530,Allan Mar Cariaso,"[0, 0]",Can't wait to try all the amazing techniques. ...,5.0,Very helpful,1393113600,"02 23, 2014"


In [ ]:
review_music = getDF("/content/drive/MyDrive/reviews_Musical_Instruments.json.gz")
review_music.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YS9MDZP93857,0006428320,John Taylor,"[0, 0]",The portfolio is fine except for the fact that...,3.0,Parts missing,1394496000,"03 11, 2014"
1,A3TS466QBAWB9D,0014072149,Silver Pencil,"[0, 0]",If you are a serious violin student on a budge...,5.0,"Perform it with a friend, today!",1370476800,"06 6, 2013"
2,A3BUDYITWUSIS7,0041291905,joyce gabriel cornett,"[0, 0]",This is and excellent edition and perfectly tr...,5.0,Vivalldi's Four Seasons,1381708800,"10 14, 2013"
3,A19K10Z0D2NTZK,0041913574,TexasCowboy,"[0, 0]",Perfect for someone who is an opera fan or a w...,5.0,Full score: voice and orchestra,1285200000,"09 23, 2010"
4,A14X336IB4JD89,0201891859,dfjm53,"[0, 1]",How many Nocturnes does it contain? All of the...,1.0,Unable to determine contents,1350432000,"10 17, 2012"



# Identifying columns with missing values

In [ ]:
def null_cols(df):
  null_cols = []
  null_perc = []
  null_records = []
  for i in df.columns:
    if df[i].isnull().sum() !=0:
      null_df = pd.DataFrame()
      null_cols.append(i)
      null_records.append(df[i].isnull().sum())
      perc = df[i].isnull().sum()/len(df[i])
      null_perc.append(perc)
  null_df = pd.DataFrame(zip(null_cols, null_records, null_perc), columns = ['Column Name', 'Total Null Records', '% of Null Records'])
  return null_df.sort_values(by = ['% of Null Records'], ascending = False)

In [ ]:
print('Home & Kitchen - Meta Data')
print()
null_cols(meta_hk)

Home & Kitchen - Meta Data



,Column Name,Total Null Records,% of Null Records
6,brand,284377,0.650766
5,price,151820,0.347424
4,related,136131,0.311521
3,description,31195,0.071386
0,salesRank,22821,0.052223
2,title,1739,0.003980
1,imUrl,371,0.000849


In [ ]:
print('Musical Instruments - Meta Data')
print()
null_cols(meta_music)

Musical Instruments - Meta Data



,Column Name,Total Null Records,% of Null Records
5,brand,50387,0.593479
4,related,16558,0.195027
1,price,15831,0.186464
0,title,9665,0.113838
6,description,6272,0.073874
3,salesRank,2368,0.027891
2,imUrl,69,0.000813


In [ ]:
print('Home & Kitchen - Reviews')
print()
null_cols(review_hk)

Home & Kitchen - Reviews



,Column Name,Total Null Records,% of Null Records
0,reviewerName,33202,0.007805


In [ ]:
print('Musical Instruments - Reviews')
print()
null_cols(review_music)

Musical Instruments - Reviews



,Column Name,Total Null Records,% of Null Records
0,reviewerName,2586,0.00517


# Cleaning Musical Instruments data

# Merging metadata and reviews for Musical Instruments category

In [ ]:
music_df = pd.merge(review_music , meta_music, on="asin", how="inner")

In [ ]:
music_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,title,price,imUrl,salesRank,categories,related,brand,description
0,A1YS9MDZP93857,0006428320,John Taylor,"[0, 0]",The portfolio is fine except for the fact that...,3.0,Parts missing,1394496000,"03 11, 2014","Six Sonatas For Two Flutes Or Violins, Volume ...",17.95,http://ecx.images-amazon.com/images/I/41EpRmh8...,{'Musical Instruments': 207315},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,NaN
1,A3TS466QBAWB9D,0014072149,Silver Pencil,"[0, 0]",If you are a serious violin student on a budge...,5.0,"Perform it with a friend, today!",1370476800,"06 6, 2013",Double Concerto in D Minor By Johann Sebastian...,18.77,http://ecx.images-amazon.com/images/I/41m6ygCq...,{'Musical Instruments': 94593},[[Musical Instruments]],"{'also_viewed': ['B0058DK7RA'], 'buy_after_vie...",,Composer: J.S. Bach.Peters Edition.For two vio...
2,A3BUDYITWUSIS7,0041291905,joyce gabriel cornett,"[0, 0]",This is and excellent edition and perfectly tr...,5.0,Vivalldi's Four Seasons,1381708800,"10 14, 2013",Hal Leonard Vivaldi Four Seasons for Piano (Or...,NaN,http://ecx.images-amazon.com/images/I/41maAqSO...,{'Musical Instruments': 222972},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,Vivaldi's famous set of four violin concertos ...
3,A19K10Z0D2NTZK,0041913574,TexasCowboy,"[0, 0]",Perfect for someone who is an opera fan or a w...,5.0,Full score: voice and orchestra,1285200000,"09 23, 2010","Aida: Opera in Quattro Atti, Partitura -- Aida...",49.99,http://ecx.images-amazon.com/images/I/513kRMv%...,NaN,[[Musical Instruments]],NaN,NaN,444 pages. \nReprint of corrected and revised ...
4,A14X336IB4JD89,0201891859,dfjm53,"[0, 1]",How many Nocturnes does it contain? All of the...,1.0,Unable to determine contents,1350432000,"10 17, 2012",Nocturnes,NaN,http://ecx.images-amazon.com/images/I/41SXCAzs...,{'Musical Instruments': 171871},"[[Musical Instruments, Instrument Accessories,...",NaN,NaN,NaN


In [ ]:
print('Music Dataframe')
print()
null_cols(music_df)

Music Dataframe



,Column Name,Total Null Records,% of Null Records
6,brand,198650,0.397160
1,title,44263,0.088495
2,price,28147,0.056274
5,related,25273,0.050528
7,description,16247,0.032483
4,salesRank,12315,0.024621
0,reviewerName,2586,0.005170
3,imUrl,438,0.000876


# Dropping unnecessary columns

In [ ]:
music_df.drop(["imUrl","related","salesRank","brand", 'categories',"reviewerName","unixReviewTime", "title","description"],axis = 1,inplace = True)

In [ ]:
music_df.head()

,reviewerID,asin,helpful,reviewText,overall,summary,reviewTime,price
0,A1YS9MDZP93857,0006428320,"[0, 0]",The portfolio is fine except for the fact that...,3.0,Parts missing,"03 11, 2014",17.95
1,A3TS466QBAWB9D,0014072149,"[0, 0]",If you are a serious violin student on a budge...,5.0,"Perform it with a friend, today!","06 6, 2013",18.77
2,A3BUDYITWUSIS7,0041291905,"[0, 0]",This is and excellent edition and perfectly tr...,5.0,Vivalldi's Four Seasons,"10 14, 2013",NaN
3,A19K10Z0D2NTZK,0041913574,"[0, 0]",Perfect for someone who is an opera fan or a w...,5.0,Full score: voice and orchestra,"09 23, 2010",49.99
4,A14X336IB4JD89,0201891859,"[0, 1]",How many Nocturnes does it contain? All of the...,1.0,Unable to determine contents,"10 17, 2012",NaN


# Data augemntation - creating helpful rate column

In [ ]:
def helpful_col(df):
  helpful_rate = []
  for x,y in df['helpful']:
    if x == 0 and y == 0:
      helpful_rate.append(float(0.000))
    else:
      helpful_rate.append(np.round((x/y),3))
  return helpful_rate

In [ ]:
music_df['helpful_rate'] = helpful_col(music_df)
music_df.drop('helpful', axis=1, inplace=True)

In [ ]:
 music_df.sort_values(['helpful_rate'], ascending=False)

,reviewerID,asin,reviewText,overall,summary,reviewTime,price,helpful_rate
250088,A2D1SZM4DR3VAI,B001R2R11Q,I just received it and have been playing it al...,5.0,Happy with my purchase,"11 30, 2013",305.00,1.0
169706,A2GGN8QETYDMU0,B000PTF0E2,I bought this mic to build video training sess...,5.0,Great mic but a little bassy until......,"02 10, 2007",61.00,1.0
169687,AJGQR9MHXWY7M,B000PTF0E2,Don't let my title completely fool you.This mi...,5.0,Great mic! Almost too good,"08 4, 2010",61.00,1.0
169689,A2D0T0KDQRRU93,B000PTF0E2,Wow!! I'm studying production of radio and tv ...,5.0,Great on my Macbook!!,"08 29, 2007",61.00,1.0
403643,A2P5W42AT3A3LP,B0061XXSVS,I bought this pedal to use with a foot tambour...,1.0,You get what you pay for,"08 23, 2013",25.46,1.0
...,...,...,...,...,...,...,...,...
212882,A8QPYIXAOXHOJ,B0018TF4TY,I'm not sure if there are other reviews for th...,5.0,Highly underrated,"01 2, 2013",NaN,0.0
212879,A2WDJOTYWG2TSD,B0018TF1LU,very good for playing metal and hard rock. awe...,5.0,B.C Rich Tribal fire warlock,"05 20, 2013",249.99,0.0
212878,A3ENA7GVYV2Y9M,B0018TF1LU,I ordered this guitar for getting into a class...,5.0,THIS GUITAR ROCKS,"09 25, 2013",249.99,0.0
212874,A39ICANSCPU2XO,B0018TF1LU,This is a breeze compared to my Ibanez. Someon...,5.0,A nice work of guitar art,"12 10, 2013",249.99,0.0


# Data formatting

In [ ]:
# change object to datetime for reviewTime column
music_df["reviewTime"] = music_df["reviewTime"].astype('datetime64[ns]')

In [ ]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500176 entries, 0 to 500175
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   reviewerID    500176 non-null  object        
 1   asin          500176 non-null  object        
 2   reviewText    500176 non-null  object        
 3   overall       500176 non-null  float64       
 4   summary       500176 non-null  object        
 5   reviewTime    500176 non-null  datetime64[ns]
 6   price         472029 non-null  float64       
 7   helpful_rate  500176 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 34.3+ MB


In [ ]:
music_df.head(10)

,reviewerID,asin,reviewText,overall,summary,reviewTime,price,helpful_rate
0,A1YS9MDZP93857,0006428320,The portfolio is fine except for the fact that...,3.0,Parts missing,2014-03-11,17.95,0.000
1,A3TS466QBAWB9D,0014072149,If you are a serious violin student on a budge...,5.0,"Perform it with a friend, today!",2013-06-06,18.77,0.000
2,A3BUDYITWUSIS7,0041291905,This is and excellent edition and perfectly tr...,5.0,Vivalldi's Four Seasons,2013-10-14,NaN,0.000
3,A19K10Z0D2NTZK,0041913574,Perfect for someone who is an opera fan or a w...,5.0,Full score: voice and orchestra,2010-09-23,49.99,0.000
4,A14X336IB4JD89,0201891859,How many Nocturnes does it contain? All of the...,1.0,Unable to determine contents,2012-10-17,NaN,0.000
5,A2HR0IL3TC4CKL,0577088726,"this was written for Carin Levine in 2008, but...",5.0,arduous indeed!,2013-06-14,NaN,0.000
6,A2DHYD72O52WS5,0634029231,Greg Koch is a knowledgable and charismatic ho...,3.0,GREAT! BUT NOT FOR BEGINNERS.,2005-06-24,NaN,0.000
7,A1MUVHT8BONL5K,0634029347,I bought this DVD and I'm returning it. The d...,2.0,NOT CLAPTION MUSIC VIDEO! A Learn How To Play...,2005-10-15,16.19,0.167
8,A15GZQZWKG6KZM,0634029347,"I was looking for a concert, not instructions ...",1.0,I was disapointed,2013-12-21,16.19,0.000
9,A16WE7UU0QD33D,0634029347,"If you are an unabashed EC fan (as I am), you ...",5.0,Solo Years Revisited,2005-10-02,16.19,0.125


# Treating Row Repetitions

Same reviewer writing the same review on the same date for the same product

In [ ]:
music_df[music_df.duplicated(subset = ['reviewText', 'reviewTime', 'asin', 'reviewerID'])]

#No duplicates exist

,reviewerID,asin,reviewText,overall,summary,reviewTime,price,helpful_rate


In [ ]:
# checking null values
null_cols(music_df)

,Column Name,Total Null Records,% of Null Records
0,price,28147,0.056274


In [ ]:
# setting date column as index 

music_df = music_df.set_index(["reviewTime"])

# Cleaning review column

In [ ]:
def review_cleaning(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    
    text = str(text).lower() #convert to lower case
    text = re.sub(r'won\'t', ' will not',text)
    text = re.sub(r'would\'t', ' would not',text)
    text = re.sub(r'could\'t', ' could not',text)
    text = re.sub(r'don\'t', ' do not',text)
    text = re.sub(r'dont', ' do not',text)
    text = re.sub(r'didn\'t', ' did not',text)
    text = re.sub(r'doesn\'t', ' does not',text)
    text = re.sub(r'\'d', ' would',text)
    text = re.sub(r'can\'t', 'can not',text)
    text = re.sub(r'n\'t',  ' not', text)
    text = re.sub(r'\'re', ' are', text)
    text = re.sub(r'\'s', ' is', text)
    text = re.sub(r'\'ll', ' will', text)
    text = re.sub(r'\'t', ' not', text)
    text = re.sub(r'\'ve', ' have', text)
    text = re.sub(r'ive', ' i have', text)
    text = re.sub(r'\'m', ' am', text)
    text = re.sub(r'im', ' i am', text)
    text = re.sub('\[.*?\]', '', text)  #remove text in square brackets
    text = re.sub('https?://\S+|www\.\S+', '', text) #removing website links
    text = re.sub('<.*?>+', '', text) #removing html tags
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #remove punctuations
    text = re.sub('\n', '', text) #removing escape character for new line
    text = re.sub('\w*\d\w*', '', text) #removing numbers between words

    return text

In [ ]:
music_df['reviewText']=music_df['reviewText'].apply(lambda x:review_cleaning(x))

In [ ]:
music_df.head()

,reviewerID,asin,reviewText,overall,summary,price,helpful_rate
reviewTime,,,,,,,
2014-03-11,A1YS9MDZP93857,0006428320,the portfolio is fine except for the fact that...,3.0,Parts missing,17.95,0.0
2013-06-06,A3TS466QBAWB9D,0014072149,if you are a serious violin student on a budge...,5.0,"Perform it with a friend, today!",18.77,0.0
2013-10-14,A3BUDYITWUSIS7,0041291905,this is and excellent edition and perfectly tr...,5.0,Vivalldi's Four Seasons,NaN,0.0
2010-09-23,A19K10Z0D2NTZK,0041913574,perfect for someone who is an opera fan or a w...,5.0,Full score: voice and orchestra,49.99,0.0
2012-10-17,A14X336IB4JD89,0201891859,how many nocturnes does it contain all of them...,1.0,Unable to determine contents,NaN,0.0


# Setting column names


In [ ]:
music_df.index.names = ['date']
music_df.rename(columns = {'reviewText': 'review',"overall" : "ratings"}, inplace=True)

In [ ]:
music_df.head()

,reviewerID,asin,review,ratings,summary,price,helpful_rate
date,,,,,,,
2014-03-11,A1YS9MDZP93857,0006428320,the portfolio is fine except for the fact that...,3.0,Parts missing,17.95,0.0
2013-06-06,A3TS466QBAWB9D,0014072149,if you are a serious violin student on a budge...,5.0,"Perform it with a friend, today!",18.77,0.0
2013-10-14,A3BUDYITWUSIS7,0041291905,this is and excellent edition and perfectly tr...,5.0,Vivalldi's Four Seasons,NaN,0.0
2010-09-23,A19K10Z0D2NTZK,0041913574,perfect for someone who is an opera fan or a w...,5.0,Full score: voice and orchestra,49.99,0.0
2012-10-17,A14X336IB4JD89,0201891859,how many nocturnes does it contain all of them...,1.0,Unable to determine contents,NaN,0.0


# Creating dataframe for sentiment analysis

In [ ]:
music_sent = pd.DataFrame(music_df['review'])
music_sent

,review
date,
2014-03-11,the portfolio is fine except for the fact that...
2013-06-06,if you are a serious violin student on a budge...
2013-10-14,this is and excellent edition and perfectly tr...
2010-09-23,perfect for someone who is an opera fan or a w...
2012-10-17,how many nocturnes does it contain all of them...
...,...
2014-07-18,i am very pleased with this recorder firstly t...
2014-07-05,i have been playing my for more years than i ...
2014-07-05,this is an exceptionally nice sounding guitar ...


## Preprocessing for sentiment analysis on music dataset


Removing the stopwords

In [ ]:
music_sent["preprocessed"]=music_sent["review"].apply(lambda x: " ".join([re.sub("[^A-Za-z]+"," ", x) for x in nltk.word_tokenize(x)]))
#returns anything which is not a letter and replaces with a space

In [ ]:
music_sent

,review,preprocessed
date,,
2014-03-11,the portfolio is fine except for the fact that...,the portfolio is fine except for the fact that...
2013-06-06,if you are a serious violin student on a budge...,if you are a serious violin student on a budge...
2013-10-14,this is and excellent edition and perfectly tr...,this is and excellent edition and perfectly tr...
2010-09-23,perfect for someone who is an opera fan or a w...,perfect for someone who is an opera fan or a w...
2012-10-17,how many nocturnes does it contain all of them...,how many nocturnes does it contain all of them...
...,...,...
2014-07-18,i am very pleased with this recorder firstly t...,i am very pleased with this recorder firstly t...
2014-07-05,i have been playing my for more years than i ...,i have been playing my for more years than i c...
2014-07-05,this is an exceptionally nice sounding guitar ...,this is an exceptionally nice sounding guitar ...


In [ ]:
stop = stopwords.words("english")
music_sent["preprocessed"]=music_sent["preprocessed"].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

# Lemmatising the reviews

In [ ]:
lemmatizer = WordNetLemmatizer() 
music_sent["preprocessed"]=music_sent["preprocessed"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [ ]:
music_sent

,review,preprocessed
date,,
2014-03-11,the portfolio is fine except for the fact that...,portfolio fine except fact last movement sonat...
2013-06-06,if you are a serious violin student on a budge...,serious violin student budget edition piano ac...
2013-10-14,this is and excellent edition and perfectly tr...,excellent edition perfectly true orchestral ve...
2010-09-23,perfect for someone who is an opera fan or a w...,perfect someone opera fan working musicianaida...
2012-10-17,how many nocturnes does it contain all of them...,many nocturne contain someunable determnine co...
...,...,...
2014-07-18,i am very pleased with this recorder firstly t...,pleased recorder firstly brand trieacutebert t...
2014-07-05,i have been playing my for more years than i ...,playing year care remember wonderful instrumen...
2014-07-05,this is an exceptionally nice sounding guitar ...,exceptionally nice sounding guitar sweet trebl...


In [ ]:
# create a function to get the subjectivity (opinionated level of text)  

def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

# create a function to get the polarity 

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

#create two new columns:

music_sent['subjectivity'] = music_sent['review'].apply(getSubjectivity)

music_sent['polarity'] = music_sent['review'].apply(getPolarity)

In [ ]:
def GetAnalysis(score):
  if score <0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

music_sent['Analysis'] = music_sent['polarity'].apply(GetAnalysis)

# Splitting into train and test

In [ ]:
music_df.head()

In [ ]:
x = music_sent["preprocessed"]
y = music_sent["Analysis"]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [ ]:
vectorizer = TfidfVectorizer(
        sublinear_tf=True, # Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).
        max_df=0.5, # ignore terms that appaer in more than that fraction of docs 
        min_df=1, #  ignore terms that have lower document frequency 
        stop_words=stopwords.words('english'),
        use_idf=True, # Enable inverse-document-frequency reweighting.
        smooth_idf=True)

In [ ]:
x_train = vectorizer.fit_transform(x_train)
print("X_train n_samples: %d, n_features: %d" % x_train.shape)

X_train n_samples: 400140, n_features: 394892


In [ ]:
x_test = vectorizer.transform(x_test)
print("X_test n_samples: %d, n_features: %d" % x_test.shape)

X_test n_samples: 100036, n_features: 394892


# Cleaning Home & Kitchen data

In [ ]:
hk_df = pd.merge(review_hk , meta_hk, on="asin", how="inner")

In [ ]:
null_cols(hk_df)

,Column Name,Total Null Records,% of Null Records
7,brand,1467701,0.345023
6,price,677444,0.159251
5,related,303183,0.071271
1,salesRank,175157,0.041175
4,description,152229,0.035786
0,reviewerName,33202,0.007805
3,title,22993,0.005405
2,imUrl,8213,0.001931


# Dropped unnecessary columns

In [ ]:
hk_df.drop(["imUrl","related","salesRank","brand", 'categories',"reviewerName","unixReviewTime", "title","description"],axis = 1,inplace = True)
hk_df.head()

# Data Augmentation - helpful_rate

In [ ]:
hk_df['helpful_rate'] = helpful_col(hk_df)
hk_df.drop('helpful', axis=1, inplace=True)

In [ ]:
hk_df.sort_values(['helpful_rate'], ascending=False)

,reviewerID,asin,reviewText,overall,summary,reviewTime,price,helpful_rate
2126963,A2BU2OLFFMPSO8,B002IAY3N0,"Works very well for a cordless vacuum in terms of cleaning power, but battery only lasted me 15 minutes or so of cleaning. It also takes a very long time to fully recharge, so it's really somethi...",3.0,"Cleans well, but very short battery life","07 8, 2010",77.99,1.0
1418468,AVJFSZ99Y12VE,B000WK1ONU,"Got this for a Samsung 60&#34; LED TV (8000 series)Build quality is excellentAll movements are well damped and stays put at any positionNo wobble or give, can support a much heavier TVCan't go wro...",5.0,Cheetah Mount,"07 6, 2013",225.00,1.0
3460018,A1IJU5D7P6WWKK,B0076PQLUK,"Perfect for what I needed, just basic chopsticks that aren't raw wood! Painted all the way through, simple and elegant.",5.0,Simple and pretty,"06 6, 2013",3.98,1.0
3460020,AL26A1F950WRG,B0076PQLUK,This set is eveb more beautiful in person. Well made and my family loves them. We have been using them a lot.,5.0,Beautiful,"04 2, 2013",3.98,1.0
1418483,A3D496YDUK46R0,B000WK1ZC0,great guality for the price;colors are awesome! my daugher had a sleep over and her friends want the same exact comforter she was glad that everyone ask her where she got it from,4.0,chea,"05 20, 2013",NaN,1.0
...,...,...,...,...,...,...,...,...
1841385,A3TFZ3WC1XYEUD,B001KYJQMI,I put two of these together and loaded them up in less than 20 minutes. No Tools required. Great Product!,5.0,Sturdy and Easy to put together,"07 15, 2013",32.00,0.0
1841387,A224CZ2E4C48WW,B001KYJQMI,"I'm taller than it. I'm 5'9&#34;. It's dead easy to assemble, and it feels like it's going to hold anything that can fit on it. I would have preferred dark gray or black, but it's in my garage,...",5.0,"For the price, it's nice","05 3, 2013",32.00,0.0
1841388,A3SVYI1V57BLIH,B001KYJQMI,"I've only had it for a couple of months, but so far it's great. Assembly is mindlessly easy. Highly Recommended...",5.0,"Great product, great price.","01 10, 2014",32.00,0.0
1841389,A1ZBK17HV7U6JD,B001KYJQMI,"this seems to be pretty good. haven't filled it up yet but seems sturdy. easy to put together, just used my hands.",4.0,seems to be pretty good,"02 23, 2014",32.00,0.0


# Data formatting

In [ ]:
# converting obj to datetime for reviewTime column
hk_df["reviewTime"] = hk_df["reviewTime"].astype('datetime64[ns]')

In [ ]:
hk_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4253926 entries, 0 to 4253925
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   reviewerID    object        
 1   asin          object        
 2   reviewText    object        
 3   overall       float64       
 4   summary       object        
 5   reviewTime    datetime64[ns]
 6   price         float64       
 7   helpful_rate  float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 292.1+ MB


In [ ]:
hk_df.head()

,reviewerID,asin,reviewText,overall,summary,reviewTime,price,helpful_rate
0,A210NOCSTBT4OD,0076144011,"Have you ever thought about how you met your best friend? Was it normal, or was it wacky - like how Elias met Shohei? Pulling a boa constrictor snake named Mathilda out of your backpack can make a...",4.0,Lovely,2012-10-04,NaN,0.000
1,A28ILV4TOG8BH2,0130350591,"The butter dish is serving us well, and keeping the butter fresh and healthy. Couldn't be happier with it, and the color is a pleasing green.",5.0,"Nice looking, and keeps the butter fresh",2011-03-22,NaN,0.000
2,A31B4D7URW4DNZ,0307394530,"I anxiously waited for the book I had pre ordered. Pics were beautiful, but...If you don't want a cake with fondant your pretty much out of luck. As most guests want something yummy to eat as wel...",2.0,Mother of the Bride,2008-06-30,14.99,0.688
3,A2HU0RPDRZZOP1,0307394530,"Fantastic book, a lot of good, original recipes. All very nicely decorated. This book will be the best gift for many.",5.0,amazing book,2010-06-24,14.99,0.000
4,A7J0XOW7DYBBD,0307394530,Can't wait to try all the amazing techniques. I just browsed and I can say that this is the book I am looking for.,5.0,Very helpful,2014-02-23,14.99,0.000


# Treating Row Repetitions

Same reviewer writing the same review on the same date for the same product

In [ ]:
#No duplicates exist
hk_df[hk_df.duplicated(subset = ['reviewText', 'reviewTime', 'asin', 'reviewerID'])]

,reviewerID,asin,reviewText,overall,summary,reviewTime,price,helpful_rate


In [ ]:
# checking null values
null_cols(hk_df)

,Column Name,Total Null Records,% of Null Records
0,price,677444,0.159251


In [ ]:
# setting date column as index 

hk_df = hk_df.set_index(["reviewTime"])

# Cleaning review column

In [ ]:
hk_df['reviewText']=hk_df['reviewText'].apply(lambda x:review_cleaning(x))

In [ ]:
hk_df.head()

,reviewerID,asin,reviewText,overall,summary,price,helpful_rate
reviewTime,,,,,,,
2012-10-04,A210NOCSTBT4OD,0076144011,have you ever thought about how you met your best friend was it normal or was it wacky like how elias met shohei pulling a boa constrictor snake named mathilda out of your backpack can make a rem...,4.0,Lovely,NaN,0.000
2011-03-22,A28ILV4TOG8BH2,0130350591,the butter dish is serving us well and keeping the butter fresh and healthy could not be happier with it and the color is a pleasing green,5.0,"Nice looking, and keeps the butter fresh",NaN,0.000
2008-06-30,A31B4D7URW4DNZ,0307394530,i anxiously waited for the book i had pre ordered pics were beautiful butif you do not want a cake with fondant your pretty much out of luck as most guests want something yummy to eat as well as...,2.0,Mother of the Bride,14.99,0.688
2010-06-24,A2HU0RPDRZZOP1,0307394530,fantastic book a lot of good original recipes all very nicely decorated this book will be the best gift for many,5.0,amazing book,14.99,0.000
2014-02-23,A7J0XOW7DYBBD,0307394530,can not wait to try all the amazing techniques i just browsed and i can say that this is the book i am looking for,5.0,Very helpful,14.99,0.000


In [ ]:
#changing column names
hk_df.index.names = ['date']
hk_df.rename(columns = {'reviewText': 'review',"overall" : "ratings"}, inplace=True)

In [ ]:
hk_df

,reviewerID,asin,review,ratings,summary,price,helpful_rate
date,,,,,,,
2012-10-04,A210NOCSTBT4OD,0076144011,have you ever thought about how you met your best friend was it normal or was it wacky like how elias met shohei pulling a boa constrictor snake named mathilda out of your backpack can make a rem...,4.0,Lovely,NaN,0.000
2011-03-22,A28ILV4TOG8BH2,0130350591,the butter dish is serving us well and keeping the butter fresh and healthy could not be happier with it and the color is a pleasing green,5.0,"Nice looking, and keeps the butter fresh",NaN,0.000
2008-06-30,A31B4D7URW4DNZ,0307394530,i anxiously waited for the book i had pre ordered pics were beautiful butif you do not want a cake with fondant your pretty much out of luck as most guests want something yummy to eat as well as...,2.0,Mother of the Bride,14.99,0.688
2010-06-24,A2HU0RPDRZZOP1,0307394530,fantastic book a lot of good original recipes all very nicely decorated this book will be the best gift for many,5.0,amazing book,14.99,0.000
2014-02-23,A7J0XOW7DYBBD,0307394530,can not wait to try all the amazing techniques i just browsed and i can say that this is the book i am looking for,5.0,Very helpful,14.99,0.000
...,...,...,...,...,...,...,...
2014-07-18,AF30DHAVSYNZ0,B00LXFP8NK,another cute set from this company that i must recommend,5.0,Cute,24.96,0.000
2014-07-21,A34KTJ4VWNBOHI,B00LXFP8NK,with a little girl who loves pink this was the perfect set for her room it is bright fun and girly with the price being so cheap we bought a couple comforter sets for the kids a wise investment,5.0,Lovely Set for Girls,24.96,0.000
2014-07-21,A4XPE5UGK51IA,B00LXFP8NK,my cousin bought this one for her daughter and she loves everything about it i like the patterns and colorscheme just perfect for a little girl,5.0,Fun Colors & Patterns,24.96,0.000


# Creating dataframe for sentiment analysis


In [ ]:
hk_sent = hk_df['review']
hk_sent

date
2012-10-04    have you ever thought about how you met your best friend was it normal or was it wacky  like how elias met shohei pulling a boa constrictor snake named mathilda out of your backpack can make a rem...
2011-03-22                                                                 the butter dish is serving us well and keeping the butter fresh and healthy could not be happier with it and the color is a pleasing green
2008-06-30    i anxiously waited for the book i had pre ordered  pics were beautiful butif you  do not want a cake with fondant your pretty much out of luck as most guests want something yummy to eat as well as...
2010-06-24                                                                                           fantastic book a lot of good original recipes all very nicely decorated this book will be the best gift for many
2014-02-23                                                                                         can not wait to try all the amazing tech

# Text Preprocessing


In [ ]:
hk_sent = pd.DataFrame(hk_df['review'])
hk_sent

,review
date,
2012-10-04,have you ever thought about how you met your best friend was it normal or was it wacky like how elias met shohei pulling a boa constrictor snake named mathilda out of your backpack can make a rem...
2011-03-22,the butter dish is serving us well and keeping the butter fresh and healthy could not be happier with it and the color is a pleasing green
2008-06-30,i anxiously waited for the book i had pre ordered pics were beautiful butif you do not want a cake with fondant your pretty much out of luck as most guests want something yummy to eat as well as...
2010-06-24,fantastic book a lot of good original recipes all very nicely decorated this book will be the best gift for many
2014-02-23,can not wait to try all the amazing techniques i just browsed and i can say that this is the book i am looking for
...,...
2014-07-18,another cute set from this company that i must recommend
2014-07-21,with a little girl who loves pink this was the perfect set for her room it is bright fun and girly with the price being so cheap we bought a couple comforter sets for the kids a wise investment
2014-07-21,my cousin bought this one for her daughter and she loves everything about it i like the patterns and colorscheme just perfect for a little girl


In [ ]:
hk_sent["preprocessed"]= hk_sent["review"].apply(lambda x: " ".join([re.sub("[^A-Za-z]+"," ", x) for x in nltk.word_tokenize(x)]))
#returns anything which is not a letter and replaces with a space


In [ ]:
hk_sent["preprocessed"]=hk_sent["preprocessed"].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))


In [ ]:
lemmatizer = WordNetLemmatizer()
hk_sent["preprocessed"]=hk_sent["preprocessed"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

# Analysing sentiment using TextBlob

In [ ]:
#create two new columns:

hk_sent['subjectivity'] = hk_sent['review'].apply(getSubjectivity)

hk_sent['polarity'] = hk_sent['review'].apply(getPolarity)

In [ ]:
hk_sent

,review,subjectivity,polarity
date,,,
2012-10-04,have you ever thought about how you met your best friend was it normal or was it wacky like how elias met shohei pulling a boa constrictor snake named mathilda out of your backpack can make a rem...,0.557619,0.294524
2011-03-22,the butter dish is serving us well and keeping the butter fresh and healthy could not be happier with it and the color is a pleasing green,0.433333,0.200000
2008-06-30,i anxiously waited for the book i had pre ordered pics were beautiful butif you do not want a cake with fondant your pretty much out of luck as most guests want something yummy to eat as well as...,0.645635,0.339286
2010-06-24,fantastic book a lot of good original recipes all very nicely decorated this book will be the best gift for many,0.675000,0.625833
2014-02-23,can not wait to try all the amazing techniques i just browsed and i can say that this is the book i am looking for,0.900000,0.600000
...,...,...,...
2014-07-18,another cute set from this company that i must recommend,1.000000,0.500000
2014-07-21,with a little girl who loves pink this was the perfect set for her room it is bright fun and girly with the price being so cheap we bought a couple comforter sets for the kids a wise investment,0.628571,0.401786
2014-07-21,my cousin bought this one for her daughter and she loves everything about it i like the patterns and colorscheme just perfect for a little girl,0.750000,0.406250


In [ ]:
hk_sent['Analysis'] = hk_sent['polarity'].apply(GetAnalysis)

In [ ]:
hk_sent['Analysis'].value_counts()

Positive    3716609
Negative     445445
Neutral       91872
Name: Analysis, dtype: int64

# Cleaned dataframes

In [ ]:
music_sent.head()

,review,preprocessed,subjectivity,polarity,Analysis
date,,,,,
2014-03-11,the portfolio is fine except for the fact that the last movement of sonata is missing what should one expect,portfolio fine except fact last movement sonata missing one expect,0.205556,0.072222,Positive
2013-06-06,if you are a serious violin student on a budget this edition has it all piano accompan i ament low price urtext solo parts and annotations by maestro david oistrakh where bach is penmanship is har...,serious violin student budget edition piano accompan ament low price urtext solo part annotation maestro david oistrakh bach penmanship hard decipher addition dash easily distinguishable original ...,0.610185,0.209259,Positive
2013-10-14,this is and excellent edition and perfectly true to the orchestral version it makes playing vivaldi a joy i uses this for a wedding and was totally satisfied with the accuracy,excellent edition perfectly true orchestral version make playing vivaldi joy us wedding totally satisfied accuracy,0.712500,0.662500,Positive
2010-09-23,perfect for someone who is an opera fan or a working musicianaida aida somet i ames spelled ada pronounced aheedah is an arabic female name meaning visitor or returning is an opera in four acts b...,perfect someone opera fan working musicianaida aida somet ames spelled ada pronounced aheedah arabic female name meaning visitor returning opera four act giuseppe verdi italian libretto antonio gh...,0.388889,0.261111,Positive
2012-10-17,how many nocturnes does it contain all of them someunable to determnine contents therefore hard to decide if this is what i needwant,many nocturne contain someunable determnine content therefore hard decide needwant,0.520833,0.104167,Positive


In [ ]:
hk_sent.head()

,review,subjectivity,polarity,Analysis
date,,,,
2012-10-04,have you ever thought about how you met your best friend was it normal or was it wacky like how elias met shohei pulling a boa constrictor snake named mathilda out of your backpack can make a rem...,0.557619,0.294524,Positive
2011-03-22,the butter dish is serving us well and keeping the butter fresh and healthy could not be happier with it and the color is a pleasing green,0.433333,0.200000,Positive
2008-06-30,i anxiously waited for the book i had pre ordered pics were beautiful butif you do not want a cake with fondant your pretty much out of luck as most guests want something yummy to eat as well as...,0.645635,0.339286,Positive
2010-06-24,fantastic book a lot of good original recipes all very nicely decorated this book will be the best gift for many,0.675000,0.625833,Positive
2014-02-23,can not wait to try all the amazing techniques i just browsed and i can say that this is the book i am looking for,0.900000,0.600000,Positive


# Exporting Preprocessed dataframes as CSV files


In [ ]:
music_sent.to_csv("music_sent.csv")

In [ ]:
hk_sent.to_csv("hk_sent.csv")